In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [2]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.engine import run_get_node
from aiida.orm import load_code, load_node
from alamode_aiida.ase_support import load_atoms
from ase import io
from tools import wait_for_node_finished

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')
TrajectoryData = DataFactory('array.trajectory')


In [3]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
print(computer_local)
code = get_or_create_code('alamode.displace_pf',
                          computer_local,
                          '/home/max/Documents/alamode/displace.py')
code_displace_pf = code
code = get_or_create_code('alamode.displace_random',
                          computer_local,
                          '/home/max/Documents/alamode/displace.py')
code_displace_random = code
code = get_or_create_code('alamode.lammps',
                         computer_local,
                         '/usr/bin/lammps')

code_lammps_name = Str('alamode.lammps@localhost')

code = get_or_create_code('alamode.extract',
                         computer_local,
                         '/home/max/Documents/alamode/tools/extract.py')
code_extract = code
code_extract_name = Str('alamode.extract@localhost')


code_displace_random, code_displace_pf, code_lammps_name


localhost (localhost), pk: 5


(<Code: Remote code 'alamode.displace_random' on localhost, pk: 59133, uuid: 9ac4bf3b-061b-4b54-b71c-720c1629204c>,
 <Code: Remote code 'alamode.displace_pf' on localhost, pk: 59132, uuid: 1edf312a-d038-4843-b752-6673dfabb07b>,
 <Str: uuid: 06171819-1bdf-4b44-9b11-484802a63700 (unstored) value: alamode.lammps@localhost>)

In [4]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


{'CWD': 'run29'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run29'

In [5]:

from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [6]:
g_prefix = nodebank.load('cubic_prefix')
g_norder = nodebank.load('cubic_norder')
g_prefix, g_norder

(<Str: uuid: c712e126-400d-4846-9bbf-772c331c7f2b (pk: 59329) value: cubic>,
 <Int: uuid: 024283bc-0bab-4a13-be6c-06e64a7c70dc (pk: 59328) value: 2>)

In [7]:
g_cwd = nodebank.load(f"{g_prefix.value}_cwd")  # ファイル保存directory
g_superstructure = nodebank.load("superstructure")  # 使用する長周期構造StructureData
g_superstructure_file = nodebank.load(
    "superstructure_file")  # 使用する長周期構造ファイルSinglefileData
g_superstructure_filepath = nodebank.load(
    "superstructure_filepath")  # 使用する長周期構造ファイルabspath
g_superstructure_filename = nodebank.load(
    "superstructure_filename")  # 使用する長周期構造ファイルfilename in the cwd directory

g_file_format = nodebank.load("structure_org_format")  # lammpsの位置フォーマット
g_file_style = nodebank.load("structure_org_style")  # lammpsの位置フォーマットのオプション
g_file_format, g_file_style, g_superstructure_file, g_cwd
g_cwd

<Str: uuid: 7311e76c-dddc-474a-88ac-81014ccec5cb (pk: 59331) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/Ga2N2_1x1x1_cubic>

## for LAMMPS

In [8]:
key = "format"
g_format = nodebank.load_or_dump(key, Str("LAMMPS"))  # alamodeのファイルオプションの定義


debug, node uuid: 0bf20c8a-c15e-4880-b961-f52d9daaddd8 (unstored) value: LAMMPS
debug, load uuid: 74e46851-89aa-44b7-be04-1be4721a28df (pk: 59136) value: LAMMPS


In [9]:
from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version
computer_local = get_or_create_local_computer('work_directory', 'localhost')
code_lammps_force = get_or_create_code('lammps.force', computer_local, 'lammps')

In [10]:
code_lammps_force_name = Str(code_lammps_force.label+"@"+'localhost')
code_lammps_force_name

<Str: uuid: 2aee468a-36bd-4016-a22e-e8ef1793fbf1 (unstored) value: lammps.force-lammps-localhost@localhost>

In [11]:
code_lammps_force.get_from_string(code_lammps_force_name.value)


<Code: Remote code 'lammps.force-lammps-localhost' on localhost, pk: 59137, uuid: 38d58d76-d1ac-419e-97e6-24a749d7c791>

In [12]:
# make lammps in and out

In [13]:
g_mag = nodebank.load_or_dump(f"{g_prefix.value}_mag", Float(0.04))


debug, node uuid: 7d5d91b1-506f-43fe-93a0-85fdc30fe55f (unstored) value: 0.04
debug, load None
debug, force dump


In [14]:

displacement_patterns = nodebank.load(f'{g_prefix.value}_suggest_pattern_folder')
displacement_patterns.list_object_names()

['cubic.pattern_ANHARM3', 'cubic.pattern_HARMONIC']

In [15]:
g_superstructure.get_ase()

Atoms(symbols='Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2Ga2N2', pbc=True, cell=[[6.38, 0.0, 0.0], [-3.19, 5.5252420761447, 0.0], [0.0, 0.0, 10.378]], masses=...)

In [16]:
g_superstructure_filename

<Str: uuid: c60d5c7e-a3c8-4b16-8707-f5fc9ed8a75b (pk: 59119) value: supercell.lammps>

In [17]:
displacement_patterns

<FolderData: uuid: 843821a8-1924-41dd-a4f1-1d2af54e7843 (pk: 59338)>

In [18]:
TrajectoryData = DataFactory('array.trajectory')
from alamode_aiida.ase_support import load_atoms_bare


In [19]:
g_action = "random"

if g_action == "pf":

    #codename = "alamode.displace_pf@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_pf

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure
    builder.structure_org_filename = g_superstructure_filename
    builder.mag = g_mag
    builder.pattern_files = displacement_patterns
        
    g_displacefuture = nodebank.load(f"{g_prefix.value}_displace_pf")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump(f"{g_prefix.value}_displace_pf", g_displacefuture)

elif g_action == "random":

    g_num_disp = nodebank.dump(f"{g_prefix.value}_num_disp", Int(40))

    #codename = "alamode.displace_random@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_random

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = g_cwd
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure
    builder.structure_org_filename = g_superstructure_filename
    builder.mag = g_mag
    builder.num_disp = g_num_disp

    g_displacefuture = nodebank.load("cubic_displace_random")
    if g_displacefuture is None:
        g_displacefuture = submit(builder)
        print(g_displacefuture)
        wait_for_node_finished(g_displacefuture)
        if g_displacefuture.is_finished_ok:
            nodebank.dump(f"{g_prefix.value}_displace_random", g_displacefuture)
        else:
            raise
else:
    raise ValueError("unknown action")


uuid: d68bcb8f-008b-4914-b5b4-eb4d2a78b4b1 (pk: 59344) (aiida.calculations:alamode.displace_random)
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
True True


In [20]:
g_displacefuture.is_finished_ok


True

In [21]:
g_displacefuture.attributes

{'job_id': '6708',
 'sealed': True,
 'version': {'core': '1.6.8'},
 'withmpi': False,
 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1},
 'append_text': '',
 'exit_status': 0,
 'parser_name': 'alamode.displace',
 'prepend_text': '',
 'last_job_info': {'job_id': '6708',
  'job_owner': 'max',
  'job_state': 'running',
  'wallclock_time_seconds': 0},
 'process_label': 'displace_random_Calcjob',
 'process_state': 'finished',
 'retrieve_list': ['displace_random.in',
  'displace_random.out',
  'cubic*.lammps',
  '_scheduler-stdout.txt',
  '_scheduler-stderr.txt'],
 'input_filename': 'displace_random.in',
 'remote_workdir': '/scratch/max/aiida/d6/8b/cb8f-008b-4914-b5b4-eb4d2a78b4b1',
 'output_filename': 'displace_random.out',
 'scheduler_state': 'done',
 'scheduler_stderr': '_scheduler-stderr.txt',
 'scheduler_stdout': '_scheduler-stdout.txt',
 'detailed_job_info': None,
 'mpirun_extra_params': [],
 'environment_variables': {},
 'import_sys_environment': True,
 'submit_script_f

In [22]:
g_displacefuture.outputs.retrieved.list_object_names()


['_scheduler-stderr.txt',
 '_scheduler-stdout.txt',
 'cubic01.lammps',
 'cubic02.lammps',
 'cubic03.lammps',
 'cubic04.lammps',
 'cubic05.lammps',
 'cubic06.lammps',
 'cubic07.lammps',
 'cubic08.lammps',
 'cubic09.lammps',
 'cubic10.lammps',
 'cubic11.lammps',
 'cubic12.lammps',
 'cubic13.lammps',
 'cubic14.lammps',
 'cubic15.lammps',
 'cubic16.lammps',
 'cubic17.lammps',
 'cubic18.lammps',
 'cubic19.lammps',
 'cubic20.lammps',
 'cubic21.lammps',
 'cubic22.lammps',
 'cubic23.lammps',
 'cubic24.lammps',
 'cubic25.lammps',
 'cubic26.lammps',
 'cubic27.lammps',
 'cubic28.lammps',
 'cubic29.lammps',
 'cubic30.lammps',
 'cubic31.lammps',
 'cubic32.lammps',
 'cubic33.lammps',
 'cubic34.lammps',
 'cubic35.lammps',
 'cubic36.lammps',
 'cubic37.lammps',
 'cubic38.lammps',
 'cubic39.lammps',
 'cubic40.lammps',
 'displace_random.out']

In [23]:
g_displace_result = g_displacefuture.outputs.results
g_displace_result


<Dict: uuid: d55d0125-26aa-4388-b035-f6888b663848 (pk: 59347)>

In [24]:
g_dispfile_folder = g_displacefuture.outputs.dispfile_folder


In [25]:
g_dispfile_folder.list_object_names()  # lammps data files


['cubic01.lammps',
 'cubic02.lammps',
 'cubic03.lammps',
 'cubic04.lammps',
 'cubic05.lammps',
 'cubic06.lammps',
 'cubic07.lammps',
 'cubic08.lammps',
 'cubic09.lammps',
 'cubic10.lammps',
 'cubic11.lammps',
 'cubic12.lammps',
 'cubic13.lammps',
 'cubic14.lammps',
 'cubic15.lammps',
 'cubic16.lammps',
 'cubic17.lammps',
 'cubic18.lammps',
 'cubic19.lammps',
 'cubic20.lammps',
 'cubic21.lammps',
 'cubic22.lammps',
 'cubic23.lammps',
 'cubic24.lammps',
 'cubic25.lammps',
 'cubic26.lammps',
 'cubic27.lammps',
 'cubic28.lammps',
 'cubic29.lammps',
 'cubic30.lammps',
 'cubic31.lammps',
 'cubic32.lammps',
 'cubic33.lammps',
 'cubic34.lammps',
 'cubic35.lammps',
 'cubic36.lammps',
 'cubic37.lammps',
 'cubic38.lammps',
 'cubic39.lammps',
 'cubic40.lammps']

In [26]:
print(g_displacefuture.outputs.displaced_structures.get_stepids())
structure = g_displacefuture.outputs.displaced_structures.get_step_structure(1)
structure.get_ase().get_positions()


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]


array([[-1.94754278e-02,  1.81610210e+00,  5.22394061e+00],
       [ 1.61634879e+00,  9.06752850e-01,  2.57497428e+00],
       [-1.62987245e-02,  1.82926833e+00,  1.99493873e+00],
       [ 1.61139467e+00,  8.87661463e-01,  4.54000253e+00],
       [ 2.16874904e-02,  1.81181974e+00,  2.65091375e-02],
       [ 1.60182208e+00,  9.44278194e-01,  7.82642497e+00],
       [-3.52253305e-02,  1.83963155e+00,  7.16844108e+00],
       [ 1.58631172e+00,  9.02986611e-01,  9.77881448e+00],
       [-1.59857942e+00,  4.64376015e+00,  5.19425609e+00],
       [-5.83249034e-03,  3.65438520e+00,  2.57890533e+00],
       [-1.55802271e+00,  4.61420197e+00,  1.97226871e+00],
       [-1.13650290e-02,  3.65999779e+00,  4.52479705e+00],
       [-1.59584850e+00,  4.59394507e+00,  4.98211198e-02],
       [ 3.24306092e-02,  3.66639129e+00,  7.77872088e+00],
       [-1.61700317e+00,  4.58062307e+00,  7.17310272e+00],
       [-2.26203604e-02,  3.70647694e+00,  9.76777482e+00],
       [ 3.18466792e+00,  1.80470416e+00

In [27]:
g_displaced_structures = g_displacefuture.outputs.displaced_structures
g_displaced_structures

<TrajectoryData: uuid: d03819ee-64ae-475c-b6c5-1f2b20fff4fc (pk: 59349)>

In [28]:
num_structures = len(g_displaced_structures.get_stepids())
num_structures

structure = g_displacefuture.outputs.displaced_structures.get_step_structure(1)


# load tersoff potential

In [29]:
import json
potentialfile = "lammps_input/potentials/tersoff.json"

with open(potentialfile,"r") as f:
    potential_dict = json.load(f)
lammps_potential = DataFactory("lammps.potential")(
    type=potential_dict["pair_style"], data=potential_dict["potential_dict"]
)
lammps_potential


<EmpiricalPotential: uuid: c00b6c0d-b4f0-40b6-af1c-e2369ee3607a (unstored)>

# execution option

In [30]:
lammps_parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': []
})
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1}
}


# run all the displacements

In [31]:
Code.get_from_string(code_lammps_force_name.value)

<Code: Remote code 'lammps.force-lammps-localhost' on localhost, pk: 59137, uuid: 38d58d76-d1ac-419e-97e6-24a749d7c791>

In [32]:
alldisp_lammps_WorkChain = WorkflowFactory("alamode.all_displace_lammps")
# alldisp_lammps_WorkChain = alldisplaces_lammps_WorkChain


In [33]:
g_prefix

<Str: uuid: c712e126-400d-4846-9bbf-772c331c7f2b (pk: 59329) value: cubic>

In [34]:
inputs = {"structures": g_displaced_structures,
          "parameters": lammps_parameters,
          "code_string":  code_lammps_force_name,
          "cwd": g_cwd,
          "prefix": g_prefix,
          'potential': lammps_potential}

#lammps_all = run(alldisp_lammps_WorkChain, **inputs)

In [35]:
#lammps_all["dfset_folder"]

In [36]:
g_cwd

<Str: uuid: 7311e76c-dddc-474a-88ac-81014ccec5cb (pk: 59331) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/Ga2N2_1x1x1_cubic>

In [37]:
# it will take 30 min. for GaN222.

In [38]:
from datetime import datetime
starttime = datetime.now()
lammps_all = nodebank.load(f"{g_prefix.value}_all_lammps")
if lammps_all is None:
    lammps_all = submit(alldisp_lammps_WorkChain, **inputs)
    print(lammps_all)
    wait_for_node_finished(lammps_all, 5)
    print(lammps_all.is_finished_ok)
    if lammps_all.is_finished_ok:
        nodebank.dump(f"{g_prefix.value}_all_lammps", lammps_all)
endtime = datetime.now()
print(endtime-starttime)

uuid: 5abb2ea9-daef-414c-a64c-8762cf5d3fe6 (pk: 59354) (aiida.workflows:alamode.all_displace_lammps)
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
True True
True
0:00:45.309880


In [39]:
lammps_all

<WorkChainNode: uuid: 5abb2ea9-daef-414c-a64c-8762cf5d3fe6 (pk: 59354) (aiida.workflows:alamode.all_displace_lammps)>

In [40]:
g_target_folder = lammps_all.outputs.dfset_folder

In [41]:
lammps_all.outputs.dfset_folder.list_object_names()

['XFSET_cubic01',
 'XFSET_cubic02',
 'XFSET_cubic03',
 'XFSET_cubic04',
 'XFSET_cubic05',
 'XFSET_cubic06',
 'XFSET_cubic07',
 'XFSET_cubic08',
 'XFSET_cubic09',
 'XFSET_cubic10',
 'XFSET_cubic11',
 'XFSET_cubic12',
 'XFSET_cubic13',
 'XFSET_cubic14',
 'XFSET_cubic15',
 'XFSET_cubic16',
 'XFSET_cubic17',
 'XFSET_cubic18',
 'XFSET_cubic19',
 'XFSET_cubic20',
 'XFSET_cubic21',
 'XFSET_cubic22',
 'XFSET_cubic23',
 'XFSET_cubic24',
 'XFSET_cubic25',
 'XFSET_cubic26',
 'XFSET_cubic27',
 'XFSET_cubic28',
 'XFSET_cubic29',
 'XFSET_cubic30',
 'XFSET_cubic31',
 'XFSET_cubic32',
 'XFSET_cubic33',
 'XFSET_cubic34',
 'XFSET_cubic35',
 'XFSET_cubic36',
 'XFSET_cubic37',
 'XFSET_cubic38',
 'XFSET_cubic39',
 'XFSET_cubic40']

In [42]:
g_prefix

<Str: uuid: c712e126-400d-4846-9bbf-772c331c7f2b (pk: 59329) value: cubic>

In [43]:
filename = lammps_all.outputs.dfset_folder.list_object_names()[0]
print(lammps_all.outputs.dfset_folder.get_object_content(filename))

ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
32
ITEM: BOX BOUNDS xy xz yz pp pp pp
-3.1899999999999999e+00 6.3799999999999999e+00 -3.1899999999999999e+00
0.0000000000000000e+00 5.5252420760999996e+00 0.0000000000000000e+00
0.0000000000000000e+00 1.0378000000000000e+01 0.0000000000000000e+00
ITEM: ATOMS element xu yu zu fx fy fz 
  Ga     0.0323676640     1.8270855399     5.1818448761    -1.3007661759     0.3187710686     1.2756605227 
  Ga     1.5953947281     0.8812787709     2.6000478100     0.0040495496     1.2696260967    -0.4323328920 
   N     0.0067592358     1.8578534918     1.9246228232     0.1474081931    -0.1298981049     1.0393038009 
   N     1.5594381107     0.9249066723     4.5372446082     1.7304170447    -0.6825017989     0.2820814096 
  Ga    -0.0391990486     1.8342312775     0.0138473643     0.7643571613     0.7808924532    -0.3807204288 
  Ga     1.5752944821     0.9023898710     7.8242088495     0.7822986007     0.8021832334    -0.5189627806 
   N     0.0133132374     1

# extract

In [44]:
code_extract_name

<Str: uuid: 97b4b57e-4a57-4bb0-8d04-f96889fc08f4 (unstored) value: alamode.extract@localhost>

In [45]:
g_superstructure_filename

<Str: uuid: c60d5c7e-a3c8-4b16-8707-f5fc9ed8a75b (pk: 59119) value: supercell.lammps>

In [46]:
code_extract = Code.get_from_string(code_extract_name.value)
code_extract

<Code: Remote code 'alamode.extract' on localhost, pk: 59135, uuid: 31808c39-5451-4302-9cea-a3288da8628d>

In [47]:
builder = code_extract.get_builder()
builder.format = g_format
builder.cwd = g_cwd
builder.structure_org_filename = g_superstructure_filename
builder.prefix = g_prefix
builder.target_folder = g_target_folder

In [48]:
extract = submit(builder)


In [49]:
wait_for_node_finished(extract)


wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
wait another 2 sec.
True True


In [50]:
nodebank.dump(f"{g_prefix.value}_extract", extract)

<CalcJobNode: uuid: e3c39776-a90c-4076-b58b-7263f75164e4 (pk: 59603) (aiida.calculations:alamode.extract)>

In [51]:
print("done")

done


In [52]:
node = nodebank.load(f'{g_prefix.value}_extract')

In [53]:
node.outputs.dfset_file.attributes

{'filename': 'DFSET_cubic'}

In [54]:
node.outputs.dfset_file.get_content()

"Namespace(VASP=None, QE=None, xTAPP=None, LAMMPS='supercell.lammps', OpenMX=None, get='disp-force', unitname='Rydberg', offset=None, emin=None, emax=None, target_file=['XFSET_cubic01', 'XFSET_cubic02', 'XFSET_cubic03', 'XFSET_cubic04', 'XFSET_cubic05', 'XFSET_cubic06', 'XFSET_cubic07', 'XFSET_cubic08', 'XFSET_cubic09', 'XFSET_cubic10', 'XFSET_cubic11', 'XFSET_cubic12', 'XFSET_cubic13', 'XFSET_cubic14', 'XFSET_cubic15', 'XFSET_cubic16', 'XFSET_cubic17', 'XFSET_cubic18', 'XFSET_cubic19', 'XFSET_cubic20', 'XFSET_cubic21', 'XFSET_cubic22', 'XFSET_cubic23', 'XFSET_cubic24', 'XFSET_cubic25', 'XFSET_cubic26', 'XFSET_cubic27', 'XFSET_cubic28', 'XFSET_cubic29', 'XFSET_cubic30', 'XFSET_cubic31', 'XFSET_cubic32', 'XFSET_cubic33', 'XFSET_cubic34', 'XFSET_cubic35', 'XFSET_cubic36', 'XFSET_cubic37', 'XFSET_cubic38', 'XFSET_cubic39', 'XFSET_cubic40'])\n# Filename: XFSET_cubic01, Snapshot: 1\n    0.06116602026581    -0.02770682205444    -0.03470956221892      -5.05917516E-02  1.23982212E-02  4.961529